# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

Reading the data from source into a pandas dataframe

In [1]:
import pandas as pd
import numpy as np

url = "https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv"

transactions_df = pd.read_csv(url)

Firstly, we have to get a concise summary of the information of the dataframe.

In [2]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20568 entries, 0 to 20567
Data columns (total 6 columns):
part       20558 non-null object
company    20568 non-null object
country    18397 non-null object
city       20535 non-null object
price      20567 non-null object
date       20568 non-null object
dtypes: object(6)
memory usage: 964.2+ KB


We can see that we have many missing information and the type of the data for each attribute is of the type 'object'. Let's print the first items of the dataset to have a better look:

In [3]:
transactions_df.head(10)

,part,company,country,city,price,date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30
3,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55
5,16729-167,Chatterbridge,Spain,Barcelona,359.52€,2016-01-02 07:20:59
6,52125-444,Voomm,France,Paris,266.62€,2016-01-02 07:40:37
7,43419-018,Buzzbean,Germany,Düsseldorf,103.45€,2016-01-02 08:57:57
8,54092-515,Zooxo,United Kingdom,London,£704.94,2016-01-02 09:09:01
9,24286-1562,Lajo,Greece,Thessaloniki,317.65€,2016-01-02 11:01:32


Obviously, the price attribute should not be an object but a number instead, so let's first extract the currency sign of the data.

In [4]:
def extract_currency(x):
    try:
        if x == '-':
            return 'NoCurrency'
        else:
            if not x[-1].isalnum():
                return x[-1]
            elif not x[0].isalnum():
                return x[0]
            else:
                return 'NoCurrency'
    except:
        return 'NoCurrency'

In [5]:
transactions_df['currency'] = transactions_df.apply(lambda x: extract_currency(x['price']),axis = 1)

Now we need to verify all the unique currencies of the transactions to check for potential invalid data:

In [6]:
currencySign = transactions_df['currency'].unique()
currencySign

array(['€', '£', '$', '¥', 'NoCurrency'], dtype=object)

We now have to take a look at the transactions labeled as 'NoCurrency'

In [7]:
transactions_df[transactions_df['currency'] == 'NoCurrency']

,part,company,country,city,price,date,currency
2414,61398-0828,Voomm,France,Paris,465.6,2016-04-15 06:51:38,NoCurrency
2415,36987-1697,Flipstorm,Greece,Athens,1266.68,2016-04-15 08:27:18,NoCurrency
2526,0603-6134,Buzzbean,Germany,Düsseldorf,829.3,2016-04-21 02:54:36,NoCurrency
2528,52380-1102,Teklist,NaN,Arnhem,357.78,2016-04-21 04:07:31,NoCurrency
8683,49349-820,Chatterbridge,Spain,Barcelona,-,2017-03-03 15:24:39,NoCurrency
8684,10267-2529,Thoughtmix,Portugal,Amadora\t,-,2017-03-03 18:07:56,NoCurrency
8685,13537-259,Ntags,Portugal,Lisbon,-,2017-03-03 19:08:54,NoCurrency
8934,68084-172,Twitterbeat,France,Annecy,void,2017-03-18 01:53:38,NoCurrency
10328,54868-0823,Chatterbridge,Spain,Barcelona,void,2017-06-02 06:49:09,NoCurrency
10329,41163-428,Avaveo,France,Nice,-,2017-06-02 07:04:35,NoCurrency


From all the transactions labeled as 'NoCurrency', we would only be interested in the ones with an actual price in the transaction column. However, those with a transaction price does not have a currency labeling the transaction value. Although we could interpret the currency of the transaction by the location, we can't be sure that the transaction was not made in a foreign currency. Therefore, we are going to clean the dataset from these invalid transactions.

In [8]:
transactions_df = transactions_df[transactions_df.currency != 'NoCurrency']

Next, we want to remove the currency sign from the transaction prices and we want to convert the signs into their respective abbreviations for clearance

In [9]:
transactions_df['price'] = transactions_df.apply(lambda x: x['price'].replace(x['currency'],''), axis=1)
currencyName = {'€': 'EUR',
                '£':'GPB',
                '$':'USD',
                '¥':'YEN'}

Now we need to convert the attributes to proper data types, i.e., prices to float numbers, dates to datetime and currencies to strings.

In [10]:
transactions_df['currency'] = transactions_df['currency'].map(currencyName).astype(str)
transactions_df['price'] = pd.to_numeric(transactions_df['price'])
transactions_df['date'] = pd.to_datetime(transactions_df['date'], errors = 'coerce')
transactions_df = transactions_df.dropna(subset=['date'], axis = 0)

Looking at the data, we can see that some transactions have the city but not a specific country. In this case, we will map the list of transactions with cities but without countries to a list of cities for each country. Also, we know that the each company is only present in one country and one city so, for the companies without a specific location, we will also map its country and city based on other entries of the same company with a valid country and city designation.

In [11]:
cityCompany = pd.Series(transactions_df['city'].values, index = transactions_df['company']).to_dict()
cityCompany.pop(' -', None)
cityCompany.pop(' a', None)
cityCompany.pop('aa', None)
cityCompany

citiesMap = pd.Series(transactions_df['country'].values, index = transactions_df['city']).to_dict()
citiesMap.pop(np.nan, None)

transactions_df['city'] = transactions_df['company'].map(cityCompany).astype(str)
transactions_df['country'] = transactions_df['city'].map(citiesMap).astype(str)

From the mapping of the country and companies, we can see that only 3 companies do not have a country specified. So, let's delete these entries from the dataset.

In [16]:
transactions_df = transactions_df[transactions_df['country'] != 'nan']

Removing entries with '\t'in the city description.

In [23]:
transactions_df['city'] = transactions_df.apply(lambda x: x['city'].replace('\t',''), axis=1)

Let's verify the unique names for companies to check for mispelling.

In [35]:
companies_list = transactions_df['company'].unique()
companies_list.sort()
companies_list

array(['Avaveo', 'Brainsphere', 'Bubblemix', 'Buzzbean', 'Chatterbridge',
       'Eimbee', 'Flipstorm', 'Gabcube', 'Gabtune', 'Gevee', 'Innojam',
       'Kanoodle', 'Laj0', 'Lajo', 'Ntags', 'Ntagz', 'Realpoint',
       'Rhycero', 'Riffpath', 'Roodel', 'Shufflebeat', 'Tagtune',
       'Teklist', 'Thoughtmix', 'Thoughtmixz', 'Twitterbeat', 'Voomm',
       'Wordify', 'Yozio', 'Zoonder', 'Zooxo', 'Zooxo.'], dtype=object)

The mispelling are:
* Laj0 and Lajo
* Ntags and Ntagz
* Thoughtmix and Thoughtmixz
* Zooxo and Zooxo.

In [36]:
transactions_df['company'] = transactions_df.apply(lambda x: x['company'].replace('Laj0','Lajo'), axis=1)
transactions_df['company'] = transactions_df.apply(lambda x: x['company'].replace('Ntagz','Ntags'), axis=1)
transactions_df['company'] = transactions_df.apply(lambda x: x['company'].replace('Thoughtmixz','Thoughtmix'), axis=1)
transactions_df['company'] = transactions_df.apply(lambda x: x['company'].replace('Zooxo.','Zooxo'), axis=1)
transactions_df['company'].unique()

array(['Chatterbridge', 'Lajo', 'Flipstorm', 'Twitterbeat', 'Voomm',
       'Buzzbean', 'Zooxo', 'Brainsphere', 'Thoughtmix', 'Wordify',
       'Teklist', 'Avaveo', 'Ntags', 'Innojam', 'Shufflebeat', 'Zoonder',
       'Kanoodle', 'Gabcube', 'Roodel', 'Riffpath', 'Eimbee', 'Yozio',
       'Rhycero', 'Realpoint', 'Gabtune', 'Bubblemix', 'Gevee', 'Tagtune'],
      dtype=object)

In [37]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20548 entries, 0 to 20567
Data columns (total 7 columns):
part        20538 non-null object
company     20548 non-null object
country     20548 non-null object
city        20548 non-null object
price       20548 non-null float64
date        20548 non-null datetime64[ns]
currency    20548 non-null object
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.3+ MB


In [38]:
transactions_df.head()

,part,company,country,city,price,date,currency
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79,2016-01-02 00:01:05,EUR
1,60505-2867,Lajo,Greece,Thessaloniki,187.99,2016-01-02 00:05:26,EUR
2,24385-268,Flipstorm,Greece,Athens,221.73,2016-01-02 00:18:30,EUR
3,76117-001,Twitterbeat,France,Annecy,1075.82,2016-01-02 02:32:30,EUR
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55,2016-01-02 04:51:55,EUR


---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

---